In [57]:
import numpy as np
import matplotlib.pyplot as plt
from time import time

from sklearn.pipeline import Pipeline


from swimnetworks import (Dense, Linear)

In [58]:
def tanh_x(x):
    """First derivative of tanh.
    """
    x = np.clip(x, -10, 10)
    return 1/np.cosh(x)**2

In [59]:
# setup the problem ODE
def u_true(x):
    return np.sin(x[:,0]) + np.sin(10*x[:,1])
def u_true_grad(x):
    return np.cos(x[:,0]) + 10*np.cos(10*x[:,1])

x_plot = np.linspace(-1*np.pi, 1*np.pi, 6000).reshape((-1, 1))
x_p, y_p = np.meshgrid(x_plot, x_plot)
# fig, ax = plt.subplots(1, 1)
# ax.plot(x_plot, u_true(x_plot), label='Solution')
# ax.plot(x_plot, u_true_grad(x_plot), label='Solution gradient')
# ax.legend()


In [60]:
activation = np.tanh
activation_x = tanh_x

def solve_swim_PINN(domain_train, u_true_train, u_grad_train, boundary_train, u_boundary_train, layer_width=20, random_state=1):
    # construct good basis functions by sampling activation functions based on the known data
    model_ansatz = Pipeline([
        ("hidden", Dense(
            layer_width=layer_width,
            activation=activation,
            parameter_sampler='tanh',
            random_seed=random_state)),
        ("linear", Linear(regularization_scale=1e-10))]
    )
    
    model_ansatz.fit(domain_train, u_grad_train)
    
    hidden_layer = model_ansatz.steps[0][1]
    linear_layer = model_ansatz.steps[1][1]

    # setup the linear system to solve for the outer coefficients
    # first, evaluate the gradient of the ansatz function
    hidden_layer.activation = activation_x
    # ux = activation_x(np.matmul(domain_train,hidden_layer.weights) + hidden_layer.biases)*hidden_layer.weights[0,:]
    # print(ux)

    ux = hidden_layer.predict(domain_train)*hidden_layer.weights[0,:]
    uy = hidden_layer.predict(domain_train)*hidden_layer.weights[1,:]
    uxy = ux+uy

    # evaluate it on the boundary as well
    hidden_layer.activation = activation
    u_boundary = hidden_layer.predict(boundary_train)

    # setup the linear system inputs and outputs
    matrix_in = np.row_stack([
        uxy,
        u_boundary
    ])
    
    # add the bias term
    matrix_in = np.column_stack([
        matrix_in,
        np.concatenate([np.zeros(uxy.shape[0]), np.ones(u_boundary.shape[0])])
    ])
    
    # construct the output matrix
    matrix_out = np.row_stack([
        u_grad_train,
        u_boundary_train
    ])

    # solve
    c = np.linalg.lstsq(matrix_in, matrix_out, rcond=1e-10)[0]
    # print(c.shape)
    linear_layer.weights = c[:-1]
    linear_layer.biases = c[-1]

    return model_ansatz

In [61]:
random_state = 1
n_pts_train = 150
lims = [-1*np.pi, 1*np.pi]

rng = np.random.default_rng(random_state)
x = rng.uniform(lims[0], lims[1], size=(n_pts_train, 1))
y = rng.uniform(lims[0], lims[1], size=(n_pts_train, 1))
x_t, y_t = np.meshgrid(x, y)

x_train = x_t.reshape(n_pts_train*n_pts_train,1)
y_train = y_t.reshape(n_pts_train*n_pts_train,1)
domain_train = np.column_stack([x_train, y_train]) 
left_boundary = np.column_stack([lims[0]*np.ones(n_pts_train), y])
right_boundary = np.column_stack([lims[1]*np.ones(n_pts_train), y])
top_boundary = np.column_stack([x, lims[1]*np.ones(n_pts_train)])
bottom_boundary = np.column_stack([x, lims[0]*np.ones(n_pts_train)])
boundary_train = np.row_stack([top_boundary, left_boundary, right_boundary, bottom_boundary])

# x_boundary = np.array(lims).reshape((-1, 1))
u_grad_train = u_true_grad(domain_train).reshape((-1,1))
u_true_train = u_true(domain_train).reshape((-1,1))
u_boundary_train = u_true(boundary_train).reshape((-1,1))

In [62]:
# fig, ax = plt.subplots()

# layer_widths = np.linspace(10, 10000, 10).astype(int)
layer_widths = 6000
t0 = time()
error = []
# for k in range(len(ax)):
model = solve_swim_PINN(domain_train, u_true_train, u_grad_train, boundary_train, u_boundary_train, layer_width=int(layer_widths), random_state=1)

# # Z = model.predict(domain_train)
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.plot_surface(x_t, y_t, u_true(domain_train),cmap='viridis', label='True')
# # ax.plot_surface(x_train, y_train, Z, cmap='viridis')
# ax.set_xlabel('X')
# ax.set_ylabel('Y')

error = np.linalg.norm(np.squeeze(model.predict(domain_train))-u_true(domain_train))
print(np.squeeze(model.predict(domain_train)))
print(u_true(domain_train))
print(error)
# print(error)
# ax.plot(x_plot, model.predict(x_plot), '--', label='Approximation')
# ax.set_title(f'width={layer_widths}')
# ax.legend()
# # error.append(np.linalg.norm(model.predict(x_plot)-u_true(x_plot)))
# print("time for solving and plotting:", time()-t0, "seconds.")
# print(error)

[-0.73461329 -0.50230519 -1.59503472 ...  0.14263432 -0.69270023
 -1.65713802]
[-0.73361489 -0.50157927 -1.59473174 ...  0.14188749 -0.69082495
 -1.65665163]
0.31092782187606904
